In [7]:
import os
import json
import time
import numpy as np
from pymongo import MongoClient
import hashlib
import argparse

np.set_printoptions(precision=3, suppress=True)

# parser = argparse.ArgumentParser()
# parser.add_argument("port", help="mongodb port to connect", type=int)
# args = parser.parse_args()

In [8]:
### Hyper-parameters
MAX_ASSIGNMENT_PER_PAPER = 50

In [9]:
dire = "../public/paper_data/title/"
paper_md5_id_list = []
paper_exp_id_list = []
paper_title_list = []
for file_name in np.sort(os.listdir(dire)):
    if file_name.startswith("[title]"):
        file = open(dire+file_name, 'r')
        title = file.read()
        file.close()
        paper_title_list.append(title)
        paper_exp_id_list.append(int(file_name[7:-4]))
        paper_md5_id_list.append(hashlib.md5(title.encode()).hexdigest())
#        print(file_name)
#        print(title)

paper_md5_id_list = np.array(paper_md5_id_list, dtype=str)
paper_title_list = np.array(paper_title_list, dtype=str)
paper_exp_id_list = np.array(paper_exp_id_list)

paper_order = np.argsort(paper_exp_id_list)
paper_md5_id_list = paper_md5_id_list[paper_order]
paper_exp_id_list = paper_exp_id_list[paper_order]
paper_title_list = paper_title_list[paper_order]

print(np.vstack((paper_exp_id_list, paper_md5_id_list, paper_title_list)).T)
#print(paper_md5_id_list)
print(len(paper_title_list))

[['0' '9a7406852e81c171d4a603dfdf94459c'
  '"In-Group Love" and "Out-Group Hate" as Motives for Individual Participation in Intergroup Conflict: A New Game Paradigm']
 ['1' '213dd60cc3d204555c05566e1b08c8a1'
  '"Walking on Eggshells": How Expressing Relationship Insecurities Perpetuates Them']
 ['2' '69e83289733fb56a7d5424c92cb77a63'
  '1/f Noise and Effort on Implicit Measures of Bias']
 ['3' 'f8774df3e8bffea0a84764dbc01d9c2e'
  'A Motion Aftereffect From Still Photographs Depicting Motion']
 ['4' '9b9f220eb1b189ec33b8d9645a6421e6'
  'A Needs-Based Model of Reconciliation: Satisfying the Differential Emotional Needs of Victim and Perpetrator as a Key to Promoting Reconciliation']
 ['5' '3d72df405d850dd7fc653d7c177fc75e'
  'A Single-System Account of the Relationship Between Priming, Recognition, and Fluency']
 ['6' 'e92b76e7d5d3be0e652d4080dcab5b42'
  'A Social Functional Approach to Emotions in Bargaining: When Communicating Anger Pays and When It Backfires']
 ['7' '05081fb9c40e685f7

In [10]:
# connect to DB:meteor and fetch the handle of collection:wageringPM
connection = MongoClient("mongodb://127.0.0.1:3001/meteor")


In [11]:
# recreate the collection
clc = connection.meteor.paper
clc.drop()
for i in range(len(paper_title_list)):
    print(paper_exp_id_list[i], paper_title_list[i])
    entry = {"paper_md5_id": paper_md5_id_list[i],
             "paper_title": paper_title_list[i], 
             "paper_exp_id": int(paper_exp_id_list[i]),
             "paper_assigned_as_first_exp_paper": int(0),
             "paper_completed_as_first_exp_paper": int(0),
             "paper_assigned": int(0),
             "paper_completed": int(0),
             "paper_max_completion": int(MAX_ASSIGNMENT_PER_PAPER) 
            }
    clc.insert_one(entry)
    
clc_hyper_para = connection.meteor.hyperparameter
clc_hyper_para.drop()
clc_hyper_para.insert_one({'max_assign_per_batch_user': 3, 'max_assign_per_user': 15})

0 "In-Group Love" and "Out-Group Hate" as Motives for Individual Participation in Intergroup Conflict: A New Game Paradigm
1 "Walking on Eggshells": How Expressing Relationship Insecurities Perpetuates Them
2 1/f Noise and Effort on Implicit Measures of Bias
3 A Motion Aftereffect From Still Photographs Depicting Motion
4 A Needs-Based Model of Reconciliation: Satisfying the Differential Emotional Needs of Victim and Perpetrator as a Key to Promoting Reconciliation
5 A Single-System Account of the Relationship Between Priming, Recognition, and Fluency
6 A Social Functional Approach to Emotions in Bargaining: When Communicating Anger Pays and When It Backfires
7 A Word-Order Constraint on Phonological Activation
8 Accelerated Relearning After Retrieval-Induced Forgetting: The Benefit of Being Forgotten
9 Accounting for Occurrences: A New View of the Use of Contingency Information in Causal Judgment
10 Across-notation Automatic Numerical Processing
11 Adaptive Memory: The Comparative Val

In [12]:
clc.drop()
for i in range(8):
    entry = {"paper_md5_id": "md5_"+str(i),
             "paper_title": "title_"+str(i), 
             "paper_exp_id": int(paper_exp_id_list[i]),
             "paper_assigned_as_first_exp_paper": int(0),
             "paper_completed_as_first_exp_paper": int(0),
             "paper_assigned": int(0),
             "paper_completed": int(0),
             "paper_max_completion": 10
            }
    clc.insert_one(entry)

clc_hyper_para = connection.meteor.hyperparameter
clc_hyper_para.drop()
clc_hyper_para.insert_one({'max_assign_per_batch_user': 3, 'max_assign_per_user': 15})